# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import numpy as np
import pandas as pd
from functools import reduce
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import random
from random import choice
import requests
import pandas as pd
from bs4 import BeautifulSoup
from scipy import stats
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [4]:
# Connection Data
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'employees'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
# Engine & Query
engine = create_engine(connection_string)
query = 'SELECT * FROM  employees_advanced'
# Database Request
df  = pd.read_sql(query,engine)


Examine the `salaries` dataset using the `head` function below.

In [5]:
# Your code here:
df.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [11]:
# Your code here:
df.isnull().sum(axis = 0)
#df['Typical Hours']
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
Name                 33183 non-null object
Job Titles           33183 non-null object
Department           33183 non-null object
Full or Part-Time    33183 non-null object
Salary or Hourly     33183 non-null object
Typical Hours        8022 non-null float64
Annual Salary        25161 non-null float64
Hourly Rate          8022 non-null float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [13]:
# Your code here:
df['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [76]:
# Your code here:
#df['Department'].value_counts()
#df.groupby('Department')['Annual Salary'].mean().sort_values(ascending=False)
df2 = pd.pivot_table(df, values=['Name'], index=['Department'], columns=['Full or Part-Time'], aggfunc='count').fillna(0)
df2

Name        
Full or Part-Time            F       P
Department                            
ADMIN HEARNG              39.0     0.0
ANIMAL CONTRL             62.0    19.0
AVIATION                1611.0    18.0
BOARD OF ELECTION        107.0     0.0
BOARD OF ETHICS            8.0     0.0
BUDGET & MGMT             46.0     0.0
BUILDINGS                269.0     0.0
BUSINESS AFFAIRS         164.0     7.0
CITY CLERK                84.0     0.0
CITY COUNCIL             360.0    51.0
COMMUNITY DEVELOPMENT    204.0     3.0
COPA                     116.0     0.0
CULTURAL AFFAIRS          65.0     0.0
DISABILITIES              28.0     0.0
DoIT                      99.0     0.0
FAMILY & SUPPORT         329.0   286.0
FINANCE                  558.0     2.0
FIRE                    4641.0     0.0
GENERAL SERVICES         980.0     0.0
HEALTH                   488.0     0.0
HUMAN RELATIONS           16.0     0.0
HUMAN RESOURCES           75.0     4.0
INSPECTOR GEN             87.0     0.0
LAW                      406.0     1.0
LICENSE APPL COMM          1.0     0.0
MAYOR'S OFFICE            84.0     1.0
OEMC                     873.0  1229.0
POLICE                 13404.0    10.0
POLICE BOARD               2.0     0.0
PROCUREMENT               92.0     0.0
PUBLIC LIBRARY           727.0   288.0
STREETS & SAN           2024.0   174.0
TRANSPORTN              1140.0     0.0
TREASURER                 22.0     0.0
WATER MGMNT             1879.0     0.0

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [58]:
# null = 30/hr
# not null != 30/hr
# we know the mean of the column is 32.7 ($/hr) (get it as hour average form hourly columns)
# n = 8022 - as n we only have 8022 hourly workers
# we know StdDEv = 12.1
# Confidence level = 95%
# t table gives us 1.96
# 32.523 | 32.7 | 32.965
# 

from scipy import stats
#stats.ttest_1samp(rvs,0.0)
import math


hourly = df.loc[df['Salary or Hourly'] == 'Hourly']['Hourly Rate']
meanX = np.mean(hourly)
sigmaX =  np.std(hourly)
conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
#We reject the null and unlikely that the mean = 30
print(conf_int)

(32.52351486943222, 33.053600563130345)


In [62]:
t, prob = stats.ttest_1samp(hourly, 30)
print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
print(f'Probablitily (p-value): {prob}')

T-stat: 20.6198057854942
Probablitily (p-value): 4.3230240486229894e-92


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [68]:
# need mean of sample, T value, StdDev
hourly = df.loc[df['Department'] == 'POLICE']['Annual Salary'].dropna().reset_index(drop=True)
meanX = np.mean(hourly)
sigmaX =  np.std(hourly)
conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
##We reject the null and unlikely that the mean = 30
print(conf_int)
t, prob = stats.ttest_1samp(hourly, 86000)
print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
print(f'Probablitily (p-value): {prob}')
# if larger than 1.86 then outside out 95% confidence interval

(86177.09579096716, 86795.73321529962)
T-stat: 3.081997005712994
Probablitily (p-value): 0.0020603403550965137


Using the `crosstab` function, find the department that has the most hourly workers. 

In [83]:
#HOW TO SORT COLUMNS
#contingency table - FreqDist of variables
pd.crosstab(df.Department, (df['Salary or Hourly']== 'Hourly'), values =df['Hourly Rate'] ,  aggfunc='count' )

Salary or Hourly,False,True
Department,,
ADMIN HEARNG,0.0,NaN
ANIMAL CONTRL,0.0,19.0
AVIATION,0.0,1082.0
BOARD OF ELECTION,0.0,NaN
BOARD OF ETHICS,0.0,NaN
BUDGET & MGMT,0.0,2.0
BUILDINGS,0.0,NaN
BUSINESS AFFAIRS,0.0,7.0
CITY CLERK,0.0,NaN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [69]:
# Your code here:

# need mean of sample, T value, StdDev
hourly = df.loc[df['Department'] == 'POLICE']['Annual Salary'].dropna().reset_index(drop=True)
meanX = np.mean(hourly)
sigmaX =  np.std(hourly)
conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
##We reject the null and unlikely that the mean = 30
print(conf_int)
t, prob = stats.ttest_1samp(hourly, 35)
print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
print(f'Probablitily (p-value): {prob}')
# if larger than 1.86 then outside out 95% confidence interval


(86177.09579096716, 86795.73321529962)
T-stat: 547.7694413343654
Probablitily (p-value): 0.0


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [93]:
# Your code here:
from scipy.stats import t
#t.interval(0.95, 10, loc=1, scale=2)
# loc is mean and scale = SD
#conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
hourly = df.loc[df['Salary or Hourly'] == 'Hourly']['Hourly Rate'].dropna().reset_index(drop=True)
meanX = np.mean(hourly)
sigmaX =  np.std(hourly)
conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
##We reject the null and unlikely that the mean = 30
print(conf_int)
t, prob = stats.ttest_1samp(hourly, 35)
print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
print(f'Probablitily (p-value): {prob}')

    





(32.52351486943222, 33.053600563130345)
T-stat: -16.352363851001144
Probablitily (p-value): 3.7636889298508017e-59


In [99]:
t.interval(0.95, hourly, meanX, sigmaX)
#why getting the error below - it worked before??????
print(f'Calculate the standard error of the mean (or standard error of measurement) of the values in the input array is {stats.sem(hourly)}')

AttributeError: 'numpy.float64' object has no attribute 'interval'

In [98]:
hourly = df.loc[df['Salary or Hourly'] == 'Hourly']['Hourly Rate'].dropna().reset_index(drop=True)
meanX = np.mean(hourly)
sigmaX =  np.std(hourly)
conf_int = stats.norm.interval(0.95, loc=meanX, scale=(sigmaX/math.sqrt(len(hourly))))
##We reject the null and unlikely that the mean = 30
print(conf_int)
t, prob = stats.ttest_1samp(hourly, 35)
print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
print(f'Probablitily (p-value): {prob}')

(32.52351486943222, 33.053600563130345)
T-stat: -16.352363851001144
Probablitily (p-value): 3.7636889298508017e-59


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [101]:
# Your code here:


# need mean of sample, T value, StdDev
annualc3 = df.loc[df['Department'] == 'POLICE']['Annual Salary'].dropna().reset_index(drop=True)
meanc3 = np.mean(annualc3)
sigmac3 =  np.std(annualc3)
conf_int = stats.norm.interval(0.95, loc=meanc3, scale=(sigmac3/math.sqrt(len(annualc3))))
##We reject the null and unlikely that the mean = 30
print(conf_int)
#t, prob = stats.ttest_1samp(hourly, 86000)
#print(f'T-stat: {t}') #t stat is how far it is in terms of StdDevs
#print(f'Probablitily (p-value): {prob}')
# if larger than 1.86 then outside out 95% confidence interval


(86177.09579096716, 86795.73321529962)


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:

